In [ ]:
import pandas as pd
import numpy as np

# Create sample data
np.random.seed(42)
dates = pd.date_range(start='2020-01-01', end='2023-12-31', freq='M')
ids = list(range(1, 101))  # 100 stocks

# Create empty lists to store data
all_dates = []
all_ids = []
all_ranks = []
all_returns = []

for date in dates:
    for id in ids:
        all_dates.append(date)
        all_ids.append(id)
        all_ranks.append(np.random.randint(1, 101))  # Random ranks between 1 and 100
        all_returns.append(np.random.normal(0.01, 0.05))  # Random returns with mean 1% and std 5%

# Create DataFrame
df = pd.DataFrame({
    'DATE': all_dates,
    'ID': all_ids,
    'rank': all_ranks,
    'returns': all_returns
})

# Initialize portfolio
portfolio = []
portfolio_returns = []
current_date = None

# Sort DataFrame by date and rank
df = df.sort_values(['DATE', 'rank'])

# Process each date
for date in df['DATE'].unique():
    date_data = df[df['DATE'] == date]
    
    if current_date is None:
        # Initial portfolio selection (top 20 ranked stocks)
        portfolio = date_data.nsmallest(20, 'rank')['ID'].tolist()
        
        # Calculate equal-weighted portfolio return
        portfolio_return = date_data[date_data['ID'].isin(portfolio)]['returns'].mean()
        portfolio_returns.append({'DATE': date, 'portfolio_return': portfolio_return})
    
    else:
        # Get current portfolio stocks' ranks
        current_portfolio_ranks = date_data[date_data['ID'].isin(portfolio)]
        
        # Remove stocks that fell below rank 50
        stocks_to_remove = current_portfolio_ranks[current_portfolio_ranks['rank'] > 50]['ID'].tolist()
        portfolio = [x for x in portfolio if x not in stocks_to_remove]
        
        # Add new top-ranked stocks to maintain 20 stocks
        available_stocks = date_data[~date_data['ID'].isin(portfolio)]
        new_stocks = available_stocks.nsmallest(20 - len(portfolio), 'rank')['ID'].tolist()
        portfolio.extend(new_stocks)
        
        # Calculate equal-weighted portfolio return
        portfolio_return = date_data[date_data['ID'].isin(portfolio)]['returns'].mean()
        portfolio_returns.append({'DATE': date, 'portfolio_return': portfolio_return})
    
    current_date = date

# Convert portfolio returns to DataFrame
portfolio_returns_df = pd.DataFrame(portfolio_returns)

# Calculate cumulative returns
portfolio_returns_df['cumulative_return'] = (1 + portfolio_returns_df['portfolio_return']).cumprod() - 1

# Plot the cumulative returns
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
plt.plot(portfolio_returns_df['DATE'], portfolio_returns_df['cumulative_return'])
plt.title('Cumulative Portfolio Returns')
plt.xlabel('Date')
plt.ylabel('Cumulative Return')
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Display summary statistics
print("\nPortfolio Return Summary Statistics:")
print(portfolio_returns_df['portfolio_return'].describe())

# Calculate annualized return and volatility
annual_return = (1 + portfolio_returns_df['portfolio_return']).prod() ** (12/len(portfolio_returns_df)) - 1
annual_volatility = portfolio_returns_df['portfolio_return'].std() * np.sqrt(12)
sharpe_ratio = annual_return / annual_volatility

print(f"\nAnnualized Return: {annual_return:.2%}")
print(f"Annualized Volatility: {annual_volatility:.2%}")
print(f"Sharpe Ratio: {sharpe_ratio:.2f}")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Recreate the sample data
np.random.seed(42)
dates = pd.date_range(start='2020-01-01', end='2023-12-31', freq='M')
ids = list(range(1, 101))  # 100 stocks

# Create empty lists to store data
all_dates = []
all_ids = []
all_ranks = []
all_returns = []

for date in dates:
    for id in ids:
        all_dates.append(date)
        all_ids.append(id)
        all_ranks.append(np.random.randint(1, 101))  # Random ranks between 1 and 100
        all_returns.append(np.random.normal(0.01, 0.05))  # Random returns with mean 1% and std 5%

# Create DataFrame
df = pd.DataFrame({
    'DATE': all_dates,
    'ID': all_ids,
    'rank': all_ranks,
    'returns': all_returns
})

# Initialize portfolio
portfolio = []
portfolio_returns = []
current_date = None

# Sort DataFrame by date and rank
df = df.sort_values(['DATE', 'rank'])

# Process each date
for date in df['DATE'].unique():
    date_data = df[df['DATE'] == date]
    
    if current_date is None:
        # Initial portfolio selection (top 20 ranked stocks)
        portfolio = date_data.nsmallest(20, 'rank')['ID'].tolist()
        
        # Calculate equal-weighted portfolio return
        portfolio_return = date_data[date_data['ID'].isin(portfolio)]['returns'].mean()
        portfolio_returns.append({'DATE': date, 'portfolio_return': portfolio_return})
    
    else:
        # Get current portfolio stocks' ranks
        current_portfolio_ranks = date_data[date_data['ID'].isin(portfolio)]
        
        # Remove stocks that fell below rank 50
        stocks_to_remove = current_portfolio_ranks[current_portfolio_ranks['rank'] > 50]['ID'].tolist()
        portfolio = [x for x in portfolio if x not in stocks_to_remove]
        
        # Add new top-ranked stocks to maintain 20 stocks
        available_stocks = date_data[~date_data['ID'].isin(portfolio)]
        new_stocks = available_stocks.nsmallest(20 - len(portfolio), 'rank')['ID'].tolist()
        portfolio.extend(new_stocks)
        
        # Calculate equal-weighted portfolio return
        portfolio_return = date_data[date_data['ID'].isin(portfolio)]['returns'].mean()
        portfolio_returns.append({'DATE': date, 'portfolio_return': portfolio_return})
    
    current_date = date

# Convert portfolio returns to DataFrame
portfolio_returns_df = pd.DataFrame(portfolio_returns)

# Calculate various performance metrics
def calculate_drawdowns(returns):
    cum_returns = (1 + returns).cumprod()
    rolling_max = cum_returns.expanding().max()
    drawdowns = (cum_returns - rolling_max) / rolling_max
    return drawdowns

def calculate_rolling_stats(returns, window=12):
    rolling_mean = returns.rolling(window=window).mean() * 12  # Annualized
    rolling_vol = returns.rolling(window=window).std() * np.sqrt(12)  # Annualized
    rolling_sharpe = rolling_mean / rolling_vol
    return rolling_mean, rolling_vol, rolling_sharpe

# Create analysis
returns = portfolio_returns_df['portfolio_return']
dates = portfolio_returns_df['DATE']

# 1. Drawdown Analysis
drawdowns = calculate_drawdowns(returns)
max_drawdown = drawdowns.min()
max_drawdown_date = dates[drawdowns.argmin()]

# 2. Rolling Statistics (12-month window)
rolling_mean, rolling_vol, rolling_sharpe = calculate_rolling_stats(returns)

# 3. Monthly Returns Heatmap
monthly_returns = returns.groupby([dates.dt.year, dates.dt.month]).mean().unstack()

# Create visualizations
plt.figure(figsize=(15, 10))

# Plot 1: Drawdown
plt.subplot(2, 2, 1)
plt.plot(dates, drawdowns)
plt.title('Drawdown')
plt.xlabel('Date')
plt.ylabel('Drawdown')
plt.grid(True)
plt.xticks(rotation=45)

# Plot 2: Rolling Statistics
plt.subplot(2, 2, 2)
plt.plot(dates, rolling_mean, label='Rolling Return (Ann.)')
plt.plot(dates, rolling_vol, label='Rolling Vol (Ann.)')
plt.title('12-Month Rolling Statistics')
plt.xlabel('Date')
plt.ylabel('Value')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)

# Plot 3: Return Distribution
plt.subplot(2, 2, 3)
plt.hist(returns, bins=30, density=True)
plt.title('Return Distribution')
plt.xlabel('Return')
plt.ylabel('Frequency')
plt.grid(True)

# Plot 4: Rolling Sharpe Ratio
plt.subplot(2, 2, 4)
plt.plot(dates, rolling_sharpe)
plt.title('12-Month Rolling Sharpe Ratio')
plt.xlabel('Date')
plt.ylabel('Sharpe Ratio')
plt.grid(True)
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

# Monthly Returns Heatmap
plt.figure(figsize=(10, 6))
sns.heatmap(monthly_returns, annot=True, fmt='.2%', cmap='RdYlGn')
plt.title('Monthly Returns Heatmap')
plt.show()

# Print Summary Statistics
print("\nPerformance Summary:")
print(f"Maximum Drawdown: {max_drawdown:.2%} (on {max_drawdown_date.strftime('%Y-%m-%d')})")
print(f"Average Monthly Return: {returns.mean():.2%}")
print(f"Monthly Return Std Dev: {returns.std():.2%}")
print(f"Best Month: {returns.max():.2%}")
print(f"Worst Month: {returns.min():.2%}")
print(f"Positive Months: {(returns > 0).sum()} ({(returns > 0).mean():.1%})")
print(f"Negative Months: {(returns < 0).sum()} ({(returns < 0).mean():.1%})")

# Calculate Value at Risk (VaR) and Conditional VaR (CVaR)
var_95 = np.percentile(returns, 5)
cvar_95 = returns[returns <= var_95].mean()
print(f"\nRisk Metrics:")
print(f"95% Value at Risk (Monthly): {var_95:.2%}")
print(f"95% Conditional VaR (Monthly): {cvar_95:.2%}")

# Calculate return metrics for different market conditions
returns_series = pd.Series(returns.values, index=dates)
rolling_std = returns_series.rolling(window=12).std()
high_vol_returns = returns_series[rolling_std > rolling_std.median()]
low_vol_returns = returns_series[rolling_std <= rolling_std.median()]

print("\nPerformance in Different Market Conditions:")
print("High Volatility Periods:")
print(f"Average Return: {high_vol_returns.mean():.2%}")
print(f"Sharpe Ratio: {high_vol_returns.mean() / high_vol_returns.std():.2f}")

print("\nLow Volatility Periods:")
print(f"Average Return: {low_vol_returns.mean():.2%}")
print(f"Sharpe Ratio: {low_vol_returns.mean() / low_vol_returns.std():.2f}")

# Calculate additional metrics
win_rate = len(returns[returns > 0]) / len(returns)
loss_rate = len(returns[returns < 0]) / len(returns)
avg_win = returns[returns > 0].mean()
avg_loss = returns[returns < 0].mean()
profit_factor = (returns[returns > 0].sum().abs() / returns[returns < 0].sum().abs())

print("\nAdditional Trading Metrics:")
print(f"Win Rate: {win_rate:.1%}")
print(f"Loss Rate: {loss_rate:.1%}")
print(f"Average Win: {avg_win:.2%}")
print(f"Average Loss: {avg_loss:.2%}")
print(f"Profit Factor: {profit_factor:.2f}")

In [ ]:
=== COMPREHENSIVE STRATEGY ANALYSIS ===

1. RETURN METRICS
Average Monthly Return: 1.05%
Annualized Return: 13.40%
Monthly Return Std Dev: 1.16%
Annualized Volatility: 4.04%
Sharpe Ratio (Annualized): 3.13

2. RISK METRICS
Maximum Drawdown: -2.27%
95% Monthly VaR: -0.74%
95% Monthly CVaR: -1.02%
Positive:Negative Months Ratio: 38:10

3. TRADING STATISTICS
Win Rate: 79.2%
Loss Rate: 20.8%
Average Winning Month: 1.47%
Average Losing Month: -0.51%
Profit Factor: 10.82
Best Month: 3.10%
Worst Month: -1.18%

4. MARKET CONDITION ANALYSIS
High Volatility Periods:
- Average Return: 0.78%
- Sharpe Ratio: 0.62
Low Volatility Periods:
- Average Return: 1.05%
- Sharpe Ratio: 0.95

5. YEARLY PERFORMANCE BREAKDOWN
      mean   std   min   max  sharpe
DATE                                
2020  1.35  1.13 -0.31  2.67  119.70
2021  1.02  1.09 -0.77  2.82   93.29
2022  1.23  1.31 -1.18  3.10   94.29
2023  0.61  1.13 -1.11  3.04   54.06